In [1]:
import json
import numpy as np
from tqdm import tqdm

In [2]:
data_train = []
with open('PIZZA_train.json', 'r') as f:
    for line in f:
        data_train.append(json.loads(line))

print(len(data_train))

2456446


In [3]:
with open('PIZZA_dev.json', 'r') as file:
    data_dev = json.load(file)

# Print the loaded data
print(len(data_dev))

348


In [4]:
data_test = []
with open('PIZZA_test.json', 'r') as f:
    for line in f:
        data_test.append(json.loads(line))

print(len(data_test))

1357


In [5]:
s=np.load('processed_sentences_no_conct_train.npy')

In [5]:
def extract_text_train(data):
    return data['train.SRC'], data['train.EXR'], data['train.TOP'], data['train.TOP-DECOUPLED']

In [6]:
def extract_text_dev(data):
    return data['dev.SRC'], data['dev.EXR'], data['dev.TOP'], data['dev.PCFG_ERR']

In [7]:
def extract_text_test(data):
    return data['test.SRC'], data['test.EXR'], data['test.TOP'], data['test.PCFG_ERR']

In [8]:
_, explanations_dev, topics_dev, decoupled_topics_dev = map(
    np.array, zip(*map(extract_text_dev, data_dev)))

In [9]:
_, explanations_train, topics_train, decoupled_topics_train = map(
    np.array, zip(*map(extract_text_train, data_train)))

In [10]:
_, explanations_test, topics_test, decoupled_topics_test = map(
    np.array, zip(*map(extract_text_test, data_test)))

In [11]:
entities_dev = set([word[1:]
               for t in topics_dev for word in t.split() if word.isupper()])

entities_train = set([word[1:]
               for t in topics_train for word in t.split() if word.isupper()])

entities_test = set([word[1:]
               for t in topics_test for word in t.split() if word.isupper()])

full_entities = entities_dev | entities_train | entities_test

enitities_exclude_not = full_entities - {'NOT'}

negate_mapping = {}
for entity in full_entities:
    if entity != 'NOT':
        negate_mapping[entity] = "NOT_" + entity


In [12]:
def negate_topic(topic):
    not_array = topic.split('NOT')
    not_array = not_array[1:]
    final_array = []
    for element in not_array:
        element = element.strip()
        open_brackets = 1
        result = ""
        index = 0
        while open_brackets > 0:
            if element[index] == '(':
                open_brackets += 1
            elif element[index] == ')':
                open_brackets -= 1
            if open_brackets > 0:
                result += element[index]
            index += 1

        final_array.append(result.strip())
    return final_array


def apply_negation(topic):
    array = negate_topic(topic)
    for el in array:
        original_el = el
        for entity in enitities_exclude_not:
            el = el.replace(entity, negate_mapping[entity])
            
        topic = topic.replace(original_el, el)
    return topic

In [13]:
negated_topics_train = np.vectorize(apply_negation)(topics_train)

In [14]:
negated_topics_dev = np.vectorize(apply_negation)(topics_dev)

In [15]:
negated_topics_test = np.vectorize(apply_negation)(topics_test)

In [27]:
final_entities = full_entities.copy()
final_entities.remove('NOT')
final_entities.remove('COMPLEX_TOPPING')
final_entities.remove('ORDER')
final_entities.remove('CONTAINERTYPE')
final_entities.remove('TOPPING')
final_entities.remove('SIZE')
final_entities.remove('QUANTITY')
final_entities.remove('NUMBER')
final_entities.remove('VOLUME')
final_entities.remove('DRINKTYPE')
final_entities.remove('STYLE')


print(final_entities)

{'PIZZAORDER', 'DRINKORDER'}


In [28]:
negate_mapping = {}
for entity in final_entities:
    if entity != 'NOT':        negate_mapping[entity] = "NOT_" + entity

print(negate_mapping)

{'PIZZAORDER': 'NOT_PIZZAORDER', 'DRINKORDER': 'NOT_DRINKORDER'}


In [29]:
final_negated_entities2 = final_entities | set(negate_mapping.values())

In [44]:
np.save("final_negated_entites2.npy",np.array(list(final_entities)))

In [20]:
final_negated_entities1 = np.load("final_negated_entites.npy")

In [21]:
final_negated_entities1

array(['QUANTITY', 'STYLE', 'SIZE', 'TOPPING', 'NOT_VOLUME',
       'NOT_TOPPING', 'NOT_SIZE', 'NUMBER', 'NOT_STYLE', 'VOLUME',
       'CONTAINERTYPE', 'NOT_CONTAINERTYPE', 'NOT_NUMBER',
       'NOT_DRINKTYPE', 'DRINKTYPE', 'NOT_QUANTITY'], dtype='<U17')

In [20]:
final_entities

{'DRINKORDER', 'PIZZAORDER'}

In [18]:
final_negated_entities2

{'DRINKORDER', 'NOT_DRINKORDER', 'NOT_PIZZAORDER', 'PIZZAORDER'}

In [31]:
negated_topics_test[2]

np.str_('(ORDER can i get (PIZZAORDER (NUMBER one ) (SIZE small ) (TOPPING pepperoni ) and (TOPPING chicken ) pizza with no (NOT (NOT_TOPPING mushrooms ) ) ) )')

In [59]:
def get_entity(topic,mapping,mapping2):
    words_array=topic.split()
        

    result=[]
    current_entity='O'
    first=False
    open_brackets=0
    for word in words_array:
        if word.startswith("("):
            if word[1:] in mapping2:
                open_brackets+=1
            else:
                if word[1:] in mapping:
                    current_entity=word[1:]
                    first=True
                elif 'NOT' not in word[1:] and 'COMPLEX_TOPPING' not in word[1:]:
                    current_entity='O'
        elif word.startswith(")"):
            if open_brackets>0:
                open_brackets-=1
            else:
                current_entity='O'
        else:
            if current_entity=='O':
                result.append(current_entity)
            else:
                if first:
                    result.append('B-'+current_entity)
                    first=False
                else:
                    result.append('I-'+current_entity)    
    return result
        


number=150
# print(s[number])
print(negated_topics_test[number])
mp=get_entity(negated_topics_test[number],final_entities,final_negated_entities1)
print(mp)
# print(tokens[500])

(ORDER get me (PIZZAORDER (NUMBER a ) (SIZE small ) (TOPPING chicken ) (TOPPING pesto ) (TOPPING tuna ) pie ) )
['O', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER']


In [48]:
for i,sen in enumerate(negated_topics_test):
    if "(NOT" in sen:
        print(i)

0
1
2
4
8
11
12
14
17
18
21
22
27
30
34
36
37
39
43
46
47
48
49
51
55
57
59
60
61
62
66
67
68
69
70
71
72
74
75
80
81
82
83
86
87
88
92
94
98
101
103
106
107
109
110
111
113
114
117
121
123
126
127
128
129
130
131
135
136
137
138
139
143
145
146
147
149
156
159
161
166
168
171
173
175
176
178
179
182
190
192
194
199
200
203
205
209
211
219
221
222
224
225
229
230
233
234
242
244
248
249
250
251
252
253
254
255
257
258
259
267
268
269
270
272
273
275
276
277
278
282
285
286
289
298
299
300
301
302
306
307
308
310
312
313
317
318
322
327
328
329
330
332
333
337
340
342
344
345
346
351
353
356
357
359
362
365
369
370
373
374
375
381
382
387
388
389
392
394
395
403
404
406
407
408
411
413
426
427
429
430
431
433
435
436
438
443
444
446
447
449
450
451
452
455
456
458
460
463
466
467
468
469
471
472
473
474
480
482
483
484
487
490
495
497
502
504
506
508
510
511
513
514
515
518
519
521
522
523
524
525
530
531
532
534
535
540
541
542
543
547
548
549
550
551
552
556
557
558
559
560
562
563
56

In [25]:
words_to_entities_train = [get_entity(topic, final_entities,final_negated_entities1) for topic in 
                           tqdm(negated_topics_train,desc="train")]
words_to_entities_dev = [get_entity(topic, final_entities,final_negated_entities1) for topic in 
                            tqdm(negated_topics_dev,desc="dev")]
words_to_entities_test = [get_entity(topic, final_entities,final_negated_entities1) for topic in
                            tqdm(negated_topics_test,desc="test")]

train:   0%|          | 0/2456446 [00:00<?, ?it/s]

test: 100%|██████████| 1357/1357 [00:00<00:00, 29713.45it/s]


In [26]:
print(len(words_to_entities_train))
print(len(words_to_entities_dev))
print(len(words_to_entities_test))
print(words_to_entities_train[0])

2456446
348
1357
['O', 'O', 'O', 'B-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER', 'I-PIZZAORDER']


In [27]:
np.save("words_to_entities_train2.npy", np.array(words_to_entities_train,dtype=object))
np.save("words_to_entities_dev2.npy", np.array(words_to_entities_dev,dtype=object))
np.save("words_to_entities_test2.npy", np.array(words_to_entities_test,dtype=object))